# บทที่ 5: การแก้ไขปัญหาและเทคนิคขั้นสูง
## Advanced ESP-NOW Troubleshooting & Optimization

![Troubleshooting](https://docs.espressif.com/projects/esp-idf/en/latest/_images/esp-now-data-flow.png)

ในบทนี้เราจะเรียนรู้การแก้ไขปัญหาขั้นสูง การ Debug และเทคนิคการใช้งาน ESP-NOW ให้มีประสิทธิภาพสูงสุด

### เป้าหมายการเรียนรู้:
- วิเคราะห์และแก้ไขปัญหาที่พบบ่อย
- เทคนิคการ Debug ขั้นสูง
- การ Monitor ประสิทธิภาพและคุณภาพสัญญาณ
- การ Optimize ESP-NOW สำหรับการใช้งานจริง
- Best Practices และ Security considerations

## ปัญหาที่พบบ่อยและวิธีแก้ไข

### ❌ ปัญหา 1: ส่งข้อมูลไม่สำเร็จ (Send Failure)

#### สาเหตุและอาการ:
- Callback `on_data_sent` ได้รับ status `ESP_NOW_SEND_FAIL`
- ข้อมูลไม่ถึงปลายทาง
- การส่งข้อมูลช้าหรือหยุดชะงัก

#### วิธีแก้ไข:

```c
// 1. ตรวจสอบ Peer ก่อนส่ง
bool is_peer_exist(const uint8_t* mac) {
    return esp_now_is_peer_exist(mac);
}

// 2. เพิ่มระบบ Retry
#define MAX_RETRY 3
typedef struct {
    uint8_t mac[6];
    uint8_t data[250];
    int len;
    int retry_count;
    uint32_t next_retry;
} retry_message_t;

static retry_message_t retry_queue[10];
static int retry_queue_size = 0;

void send_with_retry(const uint8_t* mac, const uint8_t* data, int len) {
    esp_err_t result = esp_now_send(mac, data, len);
    
    if (result != ESP_OK && retry_queue_size < 10) {
        // เพิ่มเข้า retry queue
        memcpy(retry_queue[retry_queue_size].mac, mac, 6);
        memcpy(retry_queue[retry_queue_size].data, data, len);
        retry_queue[retry_queue_size].len = len;
        retry_queue[retry_queue_size].retry_count = 0;
        retry_queue[retry_queue_size].next_retry = esp_timer_get_time()/1000 + 500;
        retry_queue_size++;
        
        ESP_LOGW(TAG, "⚠️ Send failed, added to retry queue");
    }
}

// 3. Task สำหรับประมวลผล Retry Queue
void retry_task(void *parameter) {
    while (1) {
        uint32_t current_time = esp_timer_get_time() / 1000;
        
        for (int i = 0; i < retry_queue_size; i++) {
            if (current_time >= retry_queue[i].next_retry) {
                if (retry_queue[i].retry_count < MAX_RETRY) {
                    ESP_LOGI(TAG, "🔄 Retrying send (attempt %d)", 
                            retry_queue[i].retry_count + 1);
                    
                    esp_err_t result = esp_now_send(retry_queue[i].mac, 
                                                   retry_queue[i].data, 
                                                   retry_queue[i].len);
                    
                    if (result == ESP_OK) {
                        // ลบออกจาก queue
                        for (int j = i; j < retry_queue_size - 1; j++) {
                            memcpy(&retry_queue[j], &retry_queue[j+1], 
                                   sizeof(retry_message_t));
                        }
                        retry_queue_size--;
                        i--;
                    } else {
                        retry_queue[i].retry_count++;
                        retry_queue[i].next_retry = current_time + (500 * retry_queue[i].retry_count);
                    }
                } else {
                    ESP_LOGE(TAG, "❌ Max retry reached, dropping message");
                    // ลบออกจาก queue
                    for (int j = i; j < retry_queue_size - 1; j++) {
                        memcpy(&retry_queue[j], &retry_queue[j+1], 
                               sizeof(retry_message_t));
                    }
                    retry_queue_size--;
                    i--;
                }
            }
        }
        
        vTaskDelay(pdMS_TO_TICKS(100));
    }
}
```

### ❌ ปัญหา 2: ข้อมูลเสียหาย (Data Corruption)

#### สาเหตุและอาการ:
- ข้อมูลที่รับไม่ตรงกับที่ส่ง
- ค่าใน struct ผิดเพี้ยน
- String มีตัวอักษรแปลกๆ

#### วิธีแก้ไข:

```c
// 1. เพิ่ม Checksum สำหรับตรวจสอบความถูกต้อง
#include "esp_crc.h"

typedef struct __attribute__((packed)) {
    char message[200];
    int counter;
    float sensor_value;
    uint32_t checksum;  // เพิ่ม checksum
} safe_data_t;

// ฟังก์ชันคำนวณ Checksum
uint32_t calculate_checksum(const safe_data_t* data) {
    return esp_crc32_le(0, (const uint8_t*)data, sizeof(safe_data_t) - sizeof(uint32_t));
}

// ฟังก์ชันส่งข้อมูลพร้อม Checksum
void send_safe_data(const uint8_t* mac, safe_data_t* data) {
    data->checksum = calculate_checksum(data);
    esp_now_send(mac, (uint8_t*)data, sizeof(safe_data_t));
    
    ESP_LOGI(TAG, "📤 Sent with checksum: 0x%08lX", data->checksum);
}

// ฟังก์ชันตรวจสอบข้อมูลที่รับ
bool verify_received_data(const safe_data_t* data) {
    uint32_t calculated = calculate_checksum(data);
    
    if (calculated == data->checksum) {
        ESP_LOGI(TAG, "✅ Data integrity verified");
        return true;
    } else {
        ESP_LOGE(TAG, "❌ Data corruption detected! Expected: 0x%08lX, Got: 0x%08lX", 
                 calculated, data->checksum);
        return false;
    }
}

// 2. ใช้ Magic Number สำหรับตรวจสอบประเภทข้อมูล
#define DATA_MAGIC 0xDEADBEEF

typedef struct __attribute__((packed)) {
    uint32_t magic;     // Magic number
    char message[196];  // ลดขนาดเพื่อเผื่อ magic และ checksum
    int counter;
    float sensor_value;
    uint32_t checksum;
} verified_data_t;

void on_data_recv_safe(const uint8_t *mac_addr, const uint8_t *data, int len) {
    if (len != sizeof(verified_data_t)) {
        ESP_LOGE(TAG, "❌ Invalid data size: %d (expected: %d)", len, sizeof(verified_data_t));
        return;
    }
    
    verified_data_t* recv_data = (verified_data_t*)data;
    
    // ตรวจสอบ Magic Number
    if (recv_data->magic != DATA_MAGIC) {
        ESP_LOGE(TAG, "❌ Invalid magic number: 0x%08lX", recv_data->magic);
        return;
    }
    
    // ตรวจสอบ Checksum
    if (!verify_received_data((safe_data_t*)recv_data)) {
        return;
    }
    
    // ข้อมูลถูกต้อง สามารถใช้งานได้
    ESP_LOGI(TAG, "📥 Valid data received: %s", recv_data->message);
}
```

### ❌ ปัญหา 3: การรบกวนสัญญาณ (Interference)

#### สาเหตุและอาการ:
- ความแรงสัญญาณ (RSSI) ต่ำ
- Packet Loss สูง
- การสื่อสารไม่เสถียร

#### วิธีแก้ไข:

```c
// 1. ระบบ Monitor คุณภาพสัญญาณ
typedef struct {
    uint8_t mac[6];
    int8_t last_rssi;
    uint32_t packet_received;
    uint32_t packet_lost;
    uint32_t last_seen;
    float success_rate;
} link_stats_t;

#define MAX_LINKS 10
static link_stats_t link_statistics[MAX_LINKS];
static int link_count = 0;

// 2. ฟังก์ชันบันทึกสถิติการเชื่อมต่อ
void update_link_stats(const uint8_t* mac, int8_t rssi, bool success) {
    int index = -1;
    
    // หา link ที่มีอยู่
    for (int i = 0; i < link_count; i++) {
        if (memcmp(link_statistics[i].mac, mac, 6) == 0) {
            index = i;
            break;
        }
    }
    
    // สร้าง link ใหม่
    if (index < 0 && link_count < MAX_LINKS) {
        index = link_count++;
        memcpy(link_statistics[index].mac, mac, 6);
        link_statistics[index].packet_received = 0;
        link_statistics[index].packet_lost = 0;
    }
    
    if (index >= 0) {
        link_statistics[index].last_rssi = rssi;
        link_statistics[index].last_seen = esp_timer_get_time() / 1000;
        
        if (success) {
            link_statistics[index].packet_received++;
        } else {
            link_statistics[index].packet_lost++;
        }
        
        uint32_t total = link_statistics[index].packet_received + 
                        link_statistics[index].packet_lost;
        if (total > 0) {
            link_statistics[index].success_rate = 
                (float)link_statistics[index].packet_received / total * 100.0f;
        }
        
        ESP_LOGI(TAG, "📊 Link stats [%02X:%02X:...]: RSSI=%d, Success=%.1f%%", 
                 mac[0], mac[1], rssi, link_statistics[index].success_rate);
    }
}

// 3. ฟังก์ชันเลือก Channel ที่ดีที่สุด
void scan_and_select_best_channel(void) {
    wifi_ap_record_t ap_records[20];
    uint16_t ap_count = 20;
    int channel_usage[14] = {0}; // Channel 1-13
    
    ESP_LOGI(TAG, "🔍 Scanning for best WiFi channel...");
    
    // Scan WiFi networks
    esp_wifi_scan_start(NULL, true);
    esp_wifi_scan_get_ap_records(&ap_count, ap_records);
    
    // นับการใช้งานแต่ละ channel
    for (int i = 0; i < ap_count; i++) {
        if (ap_records[i].primary >= 1 && ap_records[i].primary <= 13) {
            channel_usage[ap_records[i].primary]++;
            ESP_LOGI(TAG, "   Found AP on channel %d (RSSI: %d)", 
                     ap_records[i].primary, ap_records[i].rssi);
        }
    }
    
    // หา channel ที่ใช้น้อยที่สุด
    int best_channel = 1;
    int min_usage = channel_usage[1];
    
    for (int i = 1; i <= 13; i++) {
        ESP_LOGI(TAG, "   Channel %d: %d APs", i, channel_usage[i]);
        if (channel_usage[i] < min_usage) {
            min_usage = channel_usage[i];
            best_channel = i;
        }
    }
    
    ESP_LOGI(TAG, "✅ Best channel: %d (usage: %d APs)", best_channel, min_usage);
    
    // เปลี่ยน channel
    esp_wifi_set_channel(best_channel, WIFI_SECOND_CHAN_NONE);
    ESP_LOGI(TAG, "📡 Channel changed to %d", best_channel);
}

// 4. ระบบปรับ TX Power แบบ Adaptive
void adjust_tx_power_based_on_rssi(int8_t rssi) {
    int8_t new_power;
    
    if (rssi > -30) {
        new_power = 8;  // 2 dBm - ใกล้มาก
    } else if (rssi > -50) {
        new_power = 20; // 5 dBm - ใกล้
    } else if (rssi > -70) {
        new_power = 44; // 11 dBm - ปานกลาง
    } else {
        new_power = 78; // 19.5 dBm - ไกล
    }
    
    esp_wifi_set_max_tx_power(new_power);
    ESP_LOGI(TAG, "⚡ TX Power adjusted to %d (RSSI: %d)", new_power/4, rssi);
}
```

## เครื่องมือ Debug ขั้นสูง

### 1. ESP-NOW Monitor Tool

```c
// ระบบ Monitor และ Log ข้อมูล ESP-NOW
#include "esp_log.h"
#include "esp_timer.h"

#define MONITOR_TAG "ESPNOW_MONITOR"

typedef struct {
    uint32_t total_sent;
    uint32_t total_received;
    uint32_t send_success;
    uint32_t send_failed;
    uint32_t bytes_sent;
    uint32_t bytes_received;
    uint32_t start_time;
} espnow_stats_t;

static espnow_stats_t monitor_stats = {0};

// เริ่มต้นระบบ Monitor
void espnow_monitor_init(void) {
    memset(&monitor_stats, 0, sizeof(espnow_stats_t));
    monitor_stats.start_time = esp_timer_get_time() / 1000000; // seconds
    ESP_LOGI(MONITOR_TAG, "📊 ESP-NOW Monitor initialized");
}

// บันทึกการส่งข้อมูล
void monitor_log_send(const uint8_t* mac, const uint8_t* data, int len, bool success) {
    monitor_stats.total_sent++;
    monitor_stats.bytes_sent += len;
    
    if (success) {
        monitor_stats.send_success++;
    } else {
        monitor_stats.send_failed++;
    }
    
    ESP_LOGI(MONITOR_TAG, "📤 SEND [%s] to %02X:%02X:...:%02X (%d bytes)", 
             success ? "✅" : "❌", mac[0], mac[1], mac[5], len);
}

// บันทึกการรับข้อมูล
void monitor_log_receive(const uint8_t* mac, const uint8_t* data, int len) {
    monitor_stats.total_received++;
    monitor_stats.bytes_received += len;
    
    ESP_LOGI(MONITOR_TAG, "📥 RECV from %02X:%02X:...:%02X (%d bytes)", 
             mac[0], mac[1], mac[5], len);
}

// แสดงสถิติ
void monitor_print_stats(void) {
    uint32_t current_time = esp_timer_get_time() / 1000000;
    uint32_t uptime = current_time - monitor_stats.start_time;
    
    float send_rate = (float)monitor_stats.send_success / 
                     (monitor_stats.total_sent > 0 ? monitor_stats.total_sent : 1) * 100.0f;
    
    ESP_LOGI(MONITOR_TAG, "📊 === ESP-NOW Statistics ===");
    ESP_LOGI(MONITOR_TAG, "   Uptime: %lu seconds", uptime);
    ESP_LOGI(MONITOR_TAG, "   Sent: %lu messages (%lu bytes)", 
             monitor_stats.total_sent, monitor_stats.bytes_sent);
    ESP_LOGI(MONITOR_TAG, "   Received: %lu messages (%lu bytes)", 
             monitor_stats.total_received, monitor_stats.bytes_received);
    ESP_LOGI(MONITOR_TAG, "   Success Rate: %.1f%% (%lu/%lu)", 
             send_rate, monitor_stats.send_success, monitor_stats.total_sent);
    ESP_LOGI(MONITOR_TAG, "   Failed: %lu", monitor_stats.send_failed);
    
    if (uptime > 0) {
        ESP_LOGI(MONITOR_TAG, "   Avg Send Rate: %.2f msg/sec", 
                 (float)monitor_stats.total_sent / uptime);
        ESP_LOGI(MONITOR_TAG, "   Avg Throughput: %.2f KB/sec", 
                 (float)monitor_stats.bytes_sent / uptime / 1024.0f);
    }
    ESP_LOGI(MONITOR_TAG, "==============================");
}

// แก้ไข Callback functions เพื่อใช้กับ Monitor
void monitored_on_data_sent(const uint8_t *mac_addr, esp_now_send_status_t status) {
    bool success = (status == ESP_NOW_SEND_SUCCESS);
    monitor_log_send(mac_addr, NULL, 0, success); // ไม่มีข้อมูล data ใน callback นี้
}

void monitored_on_data_recv(const uint8_t *mac_addr, const uint8_t *data, int len) {
    monitor_log_receive(mac_addr, data, len);
    
    // เรียก callback function เดิม
    // original_on_data_recv(mac_addr, data, len);
}
```

### 2. Network Topology Analyzer

```c
// เครื่องมือวิเคราะห์โครงสร้างเครือข่าย ESP-NOW
#define MAX_NETWORK_NODES 50

typedef struct {
    uint8_t mac[6];
    char node_id[20];
    int8_t rssi;
    uint32_t last_seen;
    uint32_t message_count;
    bool is_active;
    // เพิ่มข้อมูลการเชื่อมต่อ
    uint8_t connected_nodes[10][6]; // MAC ของ nodes ที่เชื่อมต่อ
    int connection_count;
} network_node_t;

static network_node_t network_topology[MAX_NETWORK_NODES];
static int topology_node_count = 0;

// ฟังก์ชันเพิ่ม/อัพเดท node ในเครือข่าย
void topology_update_node(const uint8_t* mac, const char* node_id, int8_t rssi) {
    int index = -1;
    
    // หา node ที่มีอยู่
    for (int i = 0; i < topology_node_count; i++) {
        if (memcmp(network_topology[i].mac, mac, 6) == 0) {
            index = i;
            break;
        }
    }
    
    // สร้าง node ใหม่
    if (index < 0 && topology_node_count < MAX_NETWORK_NODES) {
        index = topology_node_count++;
        memcpy(network_topology[index].mac, mac, 6);
        network_topology[index].message_count = 0;
        network_topology[index].connection_count = 0;
    }
    
    if (index >= 0) {
        if (node_id) {
            strncpy(network_topology[index].node_id, node_id, 19);
        }
        network_topology[index].rssi = rssi;
        network_topology[index].last_seen = esp_timer_get_time() / 1000;
        network_topology[index].message_count++;
        network_topology[index].is_active = true;
    }
}

// ฟังก์ชันเพิ่มการเชื่อมต่อระหว่าง nodes
void topology_add_connection(const uint8_t* node1_mac, const uint8_t* node2_mac) {
    for (int i = 0; i < topology_node_count; i++) {
        if (memcmp(network_topology[i].mac, node1_mac, 6) == 0) {
            // ตรวจสอบว่ามีการเชื่อมต่อนี้อยู่แล้วหรือไม่
            bool exists = false;
            for (int j = 0; j < network_topology[i].connection_count; j++) {
                if (memcmp(network_topology[i].connected_nodes[j], node2_mac, 6) == 0) {
                    exists = true;
                    break;
                }
            }
            
            // เพิ่มการเชื่อมต่อใหม่
            if (!exists && network_topology[i].connection_count < 10) {
                memcpy(network_topology[i].connected_nodes[network_topology[i].connection_count], 
                       node2_mac, 6);
                network_topology[i].connection_count++;
            }
            break;
        }
    }
}

// ฟังก์ชันแสดงโครงสร้างเครือข่าย
void topology_print_network(void) {
    ESP_LOGI(MONITOR_TAG, "🕸️  === Network Topology ===");
    ESP_LOGI(MONITOR_TAG, "Total Nodes: %d", topology_node_count);
    
    for (int i = 0; i < topology_node_count; i++) {
        network_node_t* node = &network_topology[i];
        
        ESP_LOGI(MONITOR_TAG, "📍 Node: %s", 
                 strlen(node->node_id) > 0 ? node->node_id : "Unknown");
        ESP_LOGI(MONITOR_TAG, "   MAC: %02X:%02X:%02X:%02X:%02X:%02X", 
                 node->mac[0], node->mac[1], node->mac[2], 
                 node->mac[3], node->mac[4], node->mac[5]);
        ESP_LOGI(MONITOR_TAG, "   RSSI: %d dBm", node->rssi);
        ESP_LOGI(MONITOR_TAG, "   Messages: %lu", node->message_count);
        ESP_LOGI(MONITOR_TAG, "   Status: %s", node->is_active ? "Active" : "Inactive");
        ESP_LOGI(MONITOR_TAG, "   Connections: %d", node->connection_count);
        
        for (int j = 0; j < node->connection_count; j++) {
            ESP_LOGI(MONITOR_TAG, "     └─ %02X:%02X:...:%02X", 
                     node->connected_nodes[j][0], 
                     node->connected_nodes[j][1], 
                     node->connected_nodes[j][5]);
        }
        ESP_LOGI(MONITOR_TAG, "");
    }
    ESP_LOGI(MONITOR_TAG, "============================");
}

// ฟังก์ชันค้นหา Route ระหว่าง nodes (สำหรับ Mesh routing)
bool find_route_to_node(const uint8_t* target_mac, uint8_t* next_hop_mac) {
    // Simple routing: หา node ที่มี connection ไปยัง target
    for (int i = 0; i < topology_node_count; i++) {
        for (int j = 0; j < network_topology[i].connection_count; j++) {
            if (memcmp(network_topology[i].connected_nodes[j], target_mac, 6) == 0) {
                memcpy(next_hop_mac, network_topology[i].mac, 6);
                return true;
            }
        }
    }
    return false;
}
```

## Security และ Encryption

### 1. การใช้งาน ESP-NOW พร้อม Encryption

```c
// ตั้งค่า ESP-NOW พร้อม Encryption
#define ESPNOW_PMK "pmk1234567890123"  // Primary Master Key (16 bytes)
#define ESPNOW_LMK "lmk1234567890123"  // Local Master Key (16 bytes)

void setup_encrypted_espnow(void) {
    // เริ่มต้น ESP-NOW
    ESP_ERROR_CHECK(esp_now_init());
    
    // ตั้งค่า PMK (Primary Master Key)
    ESP_ERROR_CHECK(esp_now_set_pmk((uint8_t*)ESPNOW_PMK));
    
    ESP_LOGI(TAG, "🔐 ESP-NOW encryption initialized");
}

// เพิ่ม Encrypted Peer
void add_encrypted_peer(const uint8_t* mac, const uint8_t* lmk) {
    esp_now_peer_info_t peer_info = {};
    
    memcpy(peer_info.peer_addr, mac, 6);
    peer_info.channel = 0;
    peer_info.encrypt = true;  // เปิดใช้งาน encryption
    memcpy(peer_info.lmk, lmk, 16);  // Local Master Key
    
    esp_err_t result = esp_now_add_peer(&peer_info);
    if (result == ESP_OK) {
        ESP_LOGI(TAG, "✅ Encrypted peer added: %02X:%02X:...:%02X", 
                 mac[0], mac[1], mac[5]);
    } else {
        ESP_LOGE(TAG, "❌ Failed to add encrypted peer: %s", esp_err_to_name(result));
    }
}

// 2. ระบบ Authentication แบบ Challenge-Response
typedef struct {
    uint32_t challenge;
    uint32_t response;
    uint32_t timestamp;
    char node_id[20];
} auth_message_t;

static uint32_t current_challenge = 0;

// สร้าง Challenge
uint32_t generate_challenge(void) {
    return esp_random(); // ใช้ hardware random generator
}

// คำนวณ Response (แบบง่าย - ใช้ XOR กับ secret key)
uint32_t calculate_response(uint32_t challenge, const char* secret_key) {
    uint32_t key_hash = 0;
    for (int i = 0; secret_key[i] != 0; i++) {
        key_hash = (key_hash << 1) ^ secret_key[i];
    }
    return challenge ^ key_hash;
}

// ส่ง Authentication Challenge
void send_auth_challenge(const uint8_t* target_mac) {
    auth_message_t auth_msg;
    current_challenge = generate_challenge();
    
    auth_msg.challenge = current_challenge;
    auth_msg.response = 0;
    auth_msg.timestamp = esp_timer_get_time() / 1000;
    strcpy(auth_msg.node_id, "AUTH_CHALLENGE");
    
    ESP_LOGI(TAG, "🔑 Sending auth challenge: 0x%08lX", current_challenge);
    esp_now_send(target_mac, (uint8_t*)&auth_msg, sizeof(auth_msg));
}

// ตอบ Authentication Challenge
void respond_to_challenge(const uint8_t* sender_mac, uint32_t challenge) {
    auth_message_t auth_response;
    const char* secret_key = "MySecretKey123"; // ควรเก็บใน secure storage
    
    auth_response.challenge = challenge;
    auth_response.response = calculate_response(challenge, secret_key);
    auth_response.timestamp = esp_timer_get_time() / 1000;
    strcpy(auth_response.node_id, "MY_NODE_ID");
    
    ESP_LOGI(TAG, "🔐 Responding to challenge with: 0x%08lX", auth_response.response);
    esp_now_send(sender_mac, (uint8_t*)&auth_response, sizeof(auth_response));
}

// ตรวจสอบ Authentication Response
bool verify_auth_response(uint32_t challenge, uint32_t response) {
    const char* secret_key = "MySecretKey123";
    uint32_t expected = calculate_response(challenge, secret_key);
    
    bool valid = (response == expected);
    ESP_LOGI(TAG, "🔍 Auth verification: %s (expected: 0x%08lX, got: 0x%08lX)", 
             valid ? "✅ VALID" : "❌ INVALID", expected, response);
    
    return valid;
}
```

## Best Practices และ Optimization

### 1. การจัดการ Memory และ Performance

```c
// 1. Object Pool สำหรับ Message Buffers
#define MESSAGE_POOL_SIZE 20

typedef struct {
    uint8_t data[250];
    int len;
    bool in_use;
} message_buffer_t;

static message_buffer_t message_pool[MESSAGE_POOL_SIZE];
static bool pool_initialized = false;

void init_message_pool(void) {
    if (!pool_initialized) {
        memset(message_pool, 0, sizeof(message_pool));
        pool_initialized = true;
        ESP_LOGI(TAG, "📦 Message pool initialized (%d buffers)", MESSAGE_POOL_SIZE);
    }
}

message_buffer_t* allocate_message_buffer(void) {
    for (int i = 0; i < MESSAGE_POOL_SIZE; i++) {
        if (!message_pool[i].in_use) {
            message_pool[i].in_use = true;
            return &message_pool[i];
        }
    }
    ESP_LOGW(TAG, "⚠️ Message pool exhausted!");
    return NULL;
}

void free_message_buffer(message_buffer_t* buffer) {
    if (buffer >= message_pool && buffer < message_pool + MESSAGE_POOL_SIZE) {
        buffer->in_use = false;
    }
}

// 2. Batch Processing สำหรับ Callback
#define CALLBACK_QUEUE_SIZE 50

typedef struct {
    uint8_t mac[6];
    uint8_t data[250];
    int len;
    bool is_send_callback;
    esp_now_send_status_t send_status;
} callback_event_t;

static callback_event_t callback_queue[CALLBACK_QUEUE_SIZE];
static int queue_head = 0, queue_tail = 0;
static SemaphoreHandle_t queue_semaphore;

void init_callback_processing(void) {
    queue_semaphore = xSemaphoreCreateBinary();
    
    // สร้าง task สำหรับประมวลผล callback
    xTaskCreate(callback_processing_task, "callback_task", 4096, NULL, 5, NULL);
    ESP_LOGI(TAG, "🔄 Callback processing task started");
}

// เพิ่ม event เข้า queue
void queue_callback_event(const uint8_t* mac, const uint8_t* data, int len, 
                          bool is_send, esp_now_send_status_t status) {
    int next_tail = (queue_tail + 1) % CALLBACK_QUEUE_SIZE;
    
    if (next_tail != queue_head) {
        memcpy(callback_queue[queue_tail].mac, mac, 6);
        if (data && len > 0) {
            memcpy(callback_queue[queue_tail].data, data, len);
            callback_queue[queue_tail].len = len;
        }
        callback_queue[queue_tail].is_send_callback = is_send;
        callback_queue[queue_tail].send_status = status;
        
        queue_tail = next_tail;
        xSemaphoreGive(queue_semaphore);
    } else {
        ESP_LOGW(TAG, "⚠️ Callback queue full, dropping event");
    }
}

// Task ประมวลผล callback events
void callback_processing_task(void* parameter) {
    while (1) {
        if (xSemaphoreTake(queue_semaphore, portMAX_DELAY)) {
            while (queue_head != queue_tail) {
                callback_event_t* event = &callback_queue[queue_head];
                
                if (event->is_send_callback) {
                    // ประมวลผล send callback
                    process_send_callback(event->mac, event->send_status);
                } else {
                    // ประมวลผล receive callback
                    process_receive_callback(event->mac, event->data, event->len);
                }
                
                queue_head = (queue_head + 1) % CALLBACK_QUEUE_SIZE;
            }
        }
    }
}

// 3. การใช้งาน FreeRTOS Queues แทน global variables
#define ESP_NOW_QUEUE_SIZE 10

typedef struct {
    uint8_t sender_mac[6];
    uint8_t data[250];
    int data_len;
} espnow_message_t;

static QueueHandle_t espnow_queue;

void init_espnow_queue(void) {
    espnow_queue = xQueueCreate(ESP_NOW_QUEUE_SIZE, sizeof(espnow_message_t));
    if (espnow_queue == NULL) {
        ESP_LOGE(TAG, "❌ Failed to create ESP-NOW queue");
    } else {
        ESP_LOGI(TAG, "📥 ESP-NOW queue created");
    }
}

// ใน receive callback
void queue_received_message(const uint8_t *mac_addr, const uint8_t *data, int len) {
    espnow_message_t message;
    
    memcpy(message.sender_mac, mac_addr, 6);
    memcpy(message.data, data, len);
    message.data_len = len;
    
    if (xQueueSend(espnow_queue, &message, 100 / portTICK_PERIOD_MS) != pdTRUE) {
        ESP_LOGW(TAG, "⚠️ Failed to queue message, queue full");
    }
}

// ในmain task
void process_espnow_messages(void) {
    espnow_message_t message;
    
    while (xQueueReceive(espnow_queue, &message, portMAX_DELAY)) {
        ESP_LOGI(TAG, "📨 Processing queued message from %02X:%02X:...:%02X", 
                 message.sender_mac[0], message.sender_mac[1], message.sender_mac[5]);
        
        // ประมวลผลข้อความ
        // ...
    }
}
```

### 2. Power Management และ Sleep Modes

```c
// การใช้งาน ESP-NOW กับ Deep Sleep
#include "esp_sleep.h"
#include "driver/rtc_io.h"

// โครงสร้างข้อมูลสำหรับ Sleep mode
typedef struct {
    uint32_t sleep_duration_ms;
    bool wake_on_message;
    uint8_t wake_pin;
} sleep_config_t;

static sleep_config_t sleep_cfg = {
    .sleep_duration_ms = 60000,  // 1 นาที
    .wake_on_message = true,
    .wake_pin = GPIO_NUM_0
};

// ฟังก์ชันเตรียมพร้อม Deep Sleep
void prepare_for_deep_sleep(void) {
    ESP_LOGI(TAG, "😴 Preparing for deep sleep...");
    
    // บันทึกข้อมูลสำคัญลงใน RTC memory
    RTC_DATA_ATTR static int sleep_count = 0;
    sleep_count++;
    
    // ปิด ESP-NOW
    esp_now_deinit();
    esp_wifi_stop();
    
    // ตั้งค่า wake up sources
    esp_sleep_enable_timer_wakeup(sleep_cfg.sleep_duration_ms * 1000ULL); // microseconds
    
    if (sleep_cfg.wake_on_message) {
        esp_sleep_enable_ext0_wakeup(sleep_cfg.wake_pin, 0); // wake on LOW
    }
    
    ESP_LOGI(TAG, "💤 Going to deep sleep for %lu ms (count: %d)", 
             sleep_cfg.sleep_duration_ms, sleep_count);
    
    esp_deep_sleep_start();
}

// ฟังก์ชันจัดการหลัง wake up
void handle_wake_up(void) {
    esp_sleep_wakeup_cause_t wakeup_reason = esp_sleep_get_wakeup_cause();
    
    switch (wakeup_reason) {
        case ESP_SLEEP_WAKEUP_TIMER:
            ESP_LOGI(TAG, "⏰ Woke up from timer");
            break;
        case ESP_SLEEP_WAKEUP_EXT0:
            ESP_LOGI(TAG, "📨 Woke up from external pin");
            break;
        default:
            ESP_LOGI(TAG, "🔌 Not a deep sleep wake up");
            break;
    }
    
    // เริ่มต้น ESP-NOW อีกครั้ง
    init_espnow();
}

// 3. Light Sleep mode สำหรับประหยัดพลังงาน
void enter_light_sleep(uint32_t duration_ms) {
    ESP_LOGI(TAG, "😪 Entering light sleep for %lu ms", duration_ms);
    
    esp_sleep_enable_timer_wakeup(duration_ms * 1000ULL);
    esp_light_sleep_start();
    
    ESP_LOGI(TAG, "😊 Woke up from light sleep");
}

// 4. Adaptive Power Management
typedef struct {
    uint32_t message_count;
    uint32_t last_message_time;
    uint32_t idle_threshold_ms;
    bool auto_sleep_enabled;
} power_manager_t;

static power_manager_t power_mgr = {
    .message_count = 0,
    .last_message_time = 0,
    .idle_threshold_ms = 30000,  // 30 วินาที
    .auto_sleep_enabled = true
};

void power_manager_update(void) {
    uint32_t current_time = esp_timer_get_time() / 1000;
    
    if (power_mgr.auto_sleep_enabled && 
        (current_time - power_mgr.last_message_time) > power_mgr.idle_threshold_ms) {
        
        ESP_LOGI(TAG, "🔋 System idle for %lu ms, entering power save mode", 
                 current_time - power_mgr.last_message_time);
        
        // ลดความถี่ CPU
        esp_pm_config_esp32_t pm_config = {
            .max_freq_mhz = 80,   // ลดจาก 240 MHz
            .min_freq_mhz = 10,   // CPU frequency ต่ำสุด
            .light_sleep_enable = true
        };
        esp_pm_configure(&pm_config);
        
        // หรือเข้า light sleep
        enter_light_sleep(5000); // sleep 5 วินาที
    }
}

void power_manager_on_message(void) {
    power_mgr.message_count++;
    power_mgr.last_message_time = esp_timer_get_time() / 1000;
    
    // ตื่นขึ้นมาจาก power save mode
    esp_pm_config_esp32_t pm_config = {
        .max_freq_mhz = 240,  // กลับมาใช้ความเร็วเต็ม
        .min_freq_mhz = 240,
        .light_sleep_enable = false
    };
    esp_pm_configure(&pm_config);
}
```

## เครื่องมือทดสอบและ Benchmarking

### 1. ESP-NOW Performance Tester

```c
// เครื่องมือทดสอบประสิทธิภาพ ESP-NOW
#include "esp_timer.h"

typedef struct {
    uint32_t test_id;
    uint32_t packet_size;
    uint32_t total_packets;
    uint32_t packets_sent;
    uint32_t packets_received;
    uint32_t start_time;
    uint32_t end_time;
    float throughput_kbps;
    float packet_loss_percent;
    int8_t avg_rssi;
} performance_test_t;

static performance_test_t current_test;

// เริ่มต้น Performance Test
void start_performance_test(uint32_t packet_size, uint32_t total_packets) {
    memset(&current_test, 0, sizeof(performance_test_t));
    
    current_test.test_id = esp_random();
    current_test.packet_size = packet_size;
    current_test.total_packets = total_packets;
    current_test.start_time = esp_timer_get_time() / 1000; // milliseconds
    
    ESP_LOGI(TAG, "🚀 Starting performance test:");
    ESP_LOGI(TAG, "   Test ID: 0x%08lX", current_test.test_id);
    ESP_LOGI(TAG, "   Packet Size: %lu bytes", packet_size);
    ESP_LOGI(TAG, "   Total Packets: %lu", total_packets);
}

// ส่ง Test Packet
void send_test_packet(const uint8_t* target_mac) {
    if (current_test.packets_sent >= current_test.total_packets) {
        return; // Test เสร็จแล้ว
    }
    
    // สร้าง test data
    uint8_t test_data[250];
    test_data[0] = 0xAA; // Magic byte
    test_data[1] = 0x55; // Magic byte
    memcpy(&test_data[2], &current_test.test_id, 4);
    memcpy(&test_data[6], &current_test.packets_sent, 4);
    
    // เติมข้อมูลแบบ pattern
    for (int i = 10; i < current_test.packet_size; i++) {
        test_data[i] = (uint8_t)(i % 256);
    }
    
    esp_now_send(target_mac, test_data, current_test.packet_size);
    current_test.packets_sent++;
    
    if (current_test.packets_sent % 100 == 0) {
        ESP_LOGI(TAG, "📤 Sent %lu/%lu packets", 
                 current_test.packets_sent, current_test.total_packets);
    }
    
    // ตรวจสอบว่า test เสร็จหรือยัง
    if (current_test.packets_sent >= current_test.total_packets) {
        vTaskDelay(pdMS_TO_TICKS(5000)); // รอ 5 วินาทีสำหรับ packet สุดท้าย
        finish_performance_test();
    }
}

// ประมวลผล Test Packet ที่รับ
void process_test_packet(const uint8_t* data, int len, int8_t rssi) {
    if (len < 10 || data[0] != 0xAA || data[1] != 0x55) {
        return; // ไม่ใช่ test packet
    }
    
    uint32_t test_id;
    uint32_t packet_num;
    memcpy(&test_id, &data[2], 4);
    memcpy(&packet_num, &data[6], 4);
    
    if (test_id == current_test.test_id) {
        current_test.packets_received++;
        current_test.avg_rssi = (current_test.avg_rssi + rssi) / 2; // Simple average
        
        if (current_test.packets_received % 100 == 0) {
            ESP_LOGI(TAG, "📥 Received %lu packets (RSSI: %d)", 
                     current_test.packets_received, rssi);
        }
    }
}

// จบ Performance Test และแสดงผล
void finish_performance_test(void) {
    current_test.end_time = esp_timer_get_time() / 1000;
    uint32_t duration_ms = current_test.end_time - current_test.start_time;
    
    // คำนวณผลลัพธ์
    if (duration_ms > 0) {
        float total_bytes = current_test.packets_received * current_test.packet_size;
        current_test.throughput_kbps = (total_bytes * 8.0f) / duration_ms; // Kbps
    }
    
    if (current_test.total_packets > 0) {
        current_test.packet_loss_percent = 
            ((float)(current_test.total_packets - current_test.packets_received) / 
             current_test.total_packets) * 100.0f;
    }
    
    // แสดงผลลัพธ์
    ESP_LOGI(TAG, "📊 === Performance Test Results ===");
    ESP_LOGI(TAG, "   Test ID: 0x%08lX", current_test.test_id);
    ESP_LOGI(TAG, "   Duration: %lu ms", duration_ms);
    ESP_LOGI(TAG, "   Packets Sent: %lu", current_test.packets_sent);
    ESP_LOGI(TAG, "   Packets Received: %lu", current_test.packets_received);
    ESP_LOGI(TAG, "   Packet Loss: %.2f%%", current_test.packet_loss_percent);
    ESP_LOGI(TAG, "   Throughput: %.2f Kbps", current_test.throughput_kbps);
    ESP_LOGI(TAG, "   Average RSSI: %d dBm", current_test.avg_rssi);
    ESP_LOGI(TAG, "==================================");
}

// Latency Test
uint32_t measure_round_trip_time(const uint8_t* target_mac) {
    typedef struct {
        uint32_t magic;
        uint32_t timestamp;
        char message[20];
    } ping_message_t;
    
    ping_message_t ping_msg;
    ping_msg.magic = 0xPING;
    ping_msg.timestamp = esp_timer_get_time() / 1000; // milliseconds
    strcpy(ping_msg.message, "PING");
    
    ESP_LOGI(TAG, "🏓 Sending ping...");
    esp_now_send(target_mac, (uint8_t*)&ping_msg, sizeof(ping_msg));
    
    // รอ response (ต้องมี logic ใน receive callback เพื่อตอบกลับ)
    // Return RTT in milliseconds
    return 0; // จะได้จาก callback
}
```

## สรุปและข้อแนะนำสำหรับโปรเจคจริง

### ✅ ESP-NOW Best Practices Checklist:

#### 🔧 การพัฒนา (Development):
- [ ] ใช้ `__attribute__((packed))` กับ struct ทุกตัว
- [ ] เพิ่ม Magic Number และ Checksum ในข้อมูลสำคัญ
- [ ] ใช้ FreeRTOS Queue แทน global variables
- [ ] ทำ Error handling ทุก ESP-NOW API call
- [ ] ใช้ Callback แยกจาก Main logic

#### 📡 การสื่อสาร (Communication):
- [ ] จำกัดขนาดข้อมูลให้ ≤ 250 bytes
- [ ] ใช้ Sequence Number ป้องกันข้อมูลซ้ำ
- [ ] มีระบบ Retry สำหรับข้อมูลสำคัญ
- [ ] Monitor RSSI และปรับ TX Power
- [ ] เลือก WiFi Channel ที่มีการรบกวนน้อย

#### 🔐 ความปลอดภัย (Security):
- [ ] ใช้ Encryption สำหรับข้อมูลสำคัญ
- [ ] มีระบบ Authentication ก่อนรับข้อมูล
- [ ] ตรวจสอบ MAC Address ของ Sender
- [ ] ใช้ Timeout สำหรับ Authentication

#### ⚡ ประสิทธิภาพ (Performance):
- [ ] ใช้ Memory Pool แทนการ allocate แบบ dynamic
- [ ] เปิดใช้ Power Management เมื่อเหมาะสม
- [ ] Monitor และ Log สถิติการใช้งาน
- [ ] ทดสอบ Throughput และ Latency

### 🚨 ข้อควรระวัง (Common Pitfalls):

1. **ไม่ตรวจสอบ Return Value**
   ```c
   // ❌ ผิด
   esp_now_send(mac, data, len);
   
   // ✅ ถูก
   esp_err_t result = esp_now_send(mac, data, len);
   if (result != ESP_OK) {
       ESP_LOGE(TAG, "Send failed: %s", esp_err_to_name(result));
   }
   ```

2. **ใช้ Blocking Operations ใน Callback**
   ```c
   // ❌ ผิด - ห้าม delay ใน callback
   void on_data_recv(const uint8_t *mac, const uint8_t *data, int len) {
       process_data(data, len);
       vTaskDelay(100); // ❌ ห้ามทำ!
   }
   
   // ✅ ถูก - ใช้ Queue
   void on_data_recv(const uint8_t *mac, const uint8_t *data, int len) {
       xQueueSend(data_queue, data, 0);
   }
   ```

3. **ไม่จัดการ Memory Alignment**
   ```c
   // ❌ อาจมีปัญหา alignment
   typedef struct {
       char name[15];
       int value;
   } data_t;
   
   // ✅ บังคับ packing
   typedef struct __attribute__((packed)) {
       char name[15];
       int value;
   } data_t;
   ```

### 🎯 การใช้งานในโปรเจคจริง:

#### 1. IoT Sensor Network:
```c
// Sensor Node
void send_sensor_data(void) {
    sensor_data_t data = {
        .node_id = SENSOR_ID,
        .temperature = read_temperature(),
        .humidity = read_humidity(),
        .battery_level = get_battery_level(),
        .timestamp = get_rtc_time()
    };
    
    send_with_retry(gateway_mac, (uint8_t*)&data, sizeof(data));
}
```

#### 2. Smart Home Control:
```c
// Controller → Device
void send_device_command(uint8_t device_id, const char* command) {
    control_message_t cmd = {
        .target_device = device_id,
        .command_type = DEVICE_CONTROL,
        .timestamp = esp_timer_get_time() / 1000
    };
    strncpy(cmd.command, command, sizeof(cmd.command) - 1);
    
    broadcast_to_group(SMART_HOME_GROUP, &cmd, sizeof(cmd));
}
```

#### 3. Emergency Alert System:
```c
// High Priority Alert
void send_emergency_alert(const char* message) {
    alert_message_t alert = {
        .priority = EMERGENCY,
        .alert_id = generate_alert_id(),
        .timestamp = get_current_time()
    };
    strncpy(alert.message, message, sizeof(alert.message) - 1);
    
    // ส่งไปทุกคน
    broadcast_to_all(&alert, sizeof(alert));
    
    // Retry หลายครั้งสำหรับข้อความสำคัญ
    for (int i = 0; i < 3; i++) {
        vTaskDelay(pdMS_TO_TICKS(1000));
        broadcast_to_all(&alert, sizeof(alert));
    }
}
```

### 📚 แหล่งข้อมูลเพิ่มเติม:

- **ESP-IDF Documentation**: https://docs.espressif.com/projects/esp-idf/en/latest/esp32/api-reference/network/esp_now.html
- **ESP32 Datasheet**: https://www.espressif.com/sites/default/files/documentation/esp32_datasheet_en.pdf
- **Community Forum**: https://www.esp32.com/
- **GitHub Examples**: https://github.com/espressif/esp-idf/tree/master/examples/wifi/espnow

---
**🎉 ยินดีด้วย! คุณได้เรียนรู้ ESP-NOW ครบถ้วนแล้ว**

ตอนนี้คุณสามารถสร้างระบบสื่อสารไร้สายด้วย ESP-NOW ได้อย่างมั่นใจ พร้อมทั้งแก้ไขปัญหาและ optimize ประสิทธิภาพสำหรับโปรเจคจริง!